# Layer 2 — Feature Exploration

Profile all features (null %, variance, correlations with LTV30).
Analyze cohort distributions and feature importance.

**Note:** This notebook is read-only for reports. It does NOT write to MD files.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import sys

ROOT = Path('.').resolve().parent
sys.path.insert(0, str(ROOT))

# Load data
DATA_DIR = ROOT / 'data'
possible_files = ['cfm_pltv_train.csv', 'cfm_pltv_train_1.csv', 'cfm_pltv_train_imoney.csv', 'clm_pltv_iamount.csv']

df = None
for fname in possible_files:
    fpath = DATA_DIR / fname
    if fpath.exists():
        df = pd.read_csv(fpath, nrows=100_000, low_memory=False)
        print(f'✅ Loaded {len(df):,} rows from {fname}')
        break

if df is None:
    raise FileNotFoundError(f"No training data found")

# Currency settings
CURRENCY = "VND"
VND_TO_USD = 24000

def convert_currency(value, to_currency="VND"):
    if to_currency == "USD":
        return value / VND_TO_USD
    return value

def format_currency(value, currency="VND"):
    if currency == "USD":
        return f"${value:,.2f}"
    return f"₫{value:,.0f}"

currency_symbol = "₫" if CURRENCY == "VND" else "$"

✅ Loaded 100,000 rows from cfm_pltv_train.csv


In [ ]:
NUMERIC = [
    'login_rows_d7', 'active_days_d7', 'loginchannel_variety_d7',
    'network_variety_d7', 'clientversion_variety_d7', 'max_level_seen_d7',
    'max_ladderscore_d7', 'games_d7', 'win_rate_d7', 'avg_game_duration_d7',
    'avg_score_d7', 'kills_d7', 'deaths_d7', 'assists_d7', 'kd_d7',
    'max_level_game_d7', 'max_ladderlevel_d7', 'rev_d7', 'txn_cnt_d7',
]

# Feature profiling with currency conversion for monetary columns
profile_df = df[NUMERIC].copy()
if CURRENCY == "USD":
    profile_df['rev_d7'] = convert_currency(profile_df['rev_d7'], "USD")

profile = profile_df.describe().T
profile['null_pct'] = (df[NUMERIC].isnull().sum() / len(df) * 100).round(1)
print("📊 Feature Profiling:")
profile[['null_pct', 'mean', 'std', 'min', '50%', 'max']]

In [ ]:
# Spearman correlation with LTV30
corr = df[NUMERIC + ['ltv30']].corr(method='spearman')['ltv30'].drop('ltv30').sort_values(ascending=False)

fig = px.bar(x=corr.values, y=corr.index, orientation='h',
             title=f'Spearman Correlation with LTV30 ({currency_symbol})',
             labels={'x': 'Spearman ρ', 'y': 'Feature'},
             color=corr.values, color_continuous_scale='RdYlGn')
fig.update_layout(height=500, yaxis=dict(autorange='reversed'))
fig.show()

print(f"\n🔝 Top 5 Features Correlated with LTV30:")
for feat, val in corr.head(5).items():
    print(f"  {feat}: {val:.3f}")

In [ ]:
# Cohort distributions
fig_ms = px.histogram(df, x='media_source', color='is_payer_30',
                      barmode='group', title='Users by Media Source')
fig_ms.show()

fig_cc = px.histogram(df, x='first_country_code', color='is_payer_30',
                      barmode='group', title='Users by Country')
fig_cc.show()

In [ ]:
# Scatter: rev_d7 vs ltv30 with currency conversion
payers = df[df['ltv30'] > 0].sample(min(5000, len(df[df['ltv30'] > 0])), random_state=42).copy()
payers['rev_d7_display'] = convert_currency(payers['rev_d7'], CURRENCY)
payers['ltv30_display'] = convert_currency(payers['ltv30'], CURRENCY)

fig = px.scatter(payers, x='rev_d7_display', y='ltv30_display', color='first_country_code',
                 title=f'D7 Revenue vs LTV30 (payers) - {CURRENCY}', opacity=0.5,
                 labels={'rev_d7_display': f'Revenue D7 ({currency_symbol})', 
                         'ltv30_display': f'LTV30 ({currency_symbol})'})
fig.show()

In [ ]:
## Summary

This notebook provides feature-level analysis including:
- Feature profiling (null %, mean, std, min/max)
- Spearman correlation with LTV30
- Cohort distributions by media source and country
- D7 revenue vs LTV30 relationship

**Note:** This is an exploratory notebook. It does NOT write to report MD files.